# 테이블 아래로 붙이기 - SET

- SET XXX YYY: XXX테이블과 YYY테이블을 연속하여 불러와서 하나의 테이블로 만듦

```SAS
DATA CLASS2;
INPUT NAME $ SEX $ AGE HEIGHT WEIGHT;
CARDS;
최철기 남 12 150 50
주양 여 11 130 40
;
RUN;

DATA TEST;
SET
SASHELP.CLASS     /* SET SASHELP.CLASS CLASS2; 이렇게 한 줄로 써도 됨 */
CLASS2;     /* SASHELP라이브러리에 있는 CLASS테이블과 위에서 만든 CLASS2테이블을 연속으로 불러와 병합 */
RUN;
```

#### 컬럼명이 일치하지 않는 경우

```SAS
DATA CLASS2;
INPUT NAME $ AGE HEIGHT WEIGHT ADDR $;
CARDS;
CHOI 12 150 50 NAMDEMUN
JOO 11 130 40 SEOCHO
;
RUN;

DATA TEST;
SET
SASHELP.CLASS     /* 해당 테이블의 컬럼은 NAME, SEX, AGE, HEIGHT, WEIGHT */
CLASS2;     /* 이 테이블의 컬럼은 NAME, AGE, HEIGHT, WEIGHT, ADDR */
RUN;     /* 즉, 두 테이블의 컬럼명이 일치하지 않고 순서도 다름. 그 결과 병합하면 두 테이블에 존재하는 모든 컬럼이 생성되고 테이블에 해당 컬럼이 없었을 경우에는 빈 칸으로 됨 */
```

#### 컬럼의 길이가 다른 경우

- 컬럼명은 똑같지만 길이가 다른 경우의 병합
- 예시: 테이블1은 SASHELP라이브러리의 CLASS테이블 사용.
- 먼저 이 테이블의 속성을 PROC CONTENTS를 통해 확인

```SAS
PROC CONTENTS DATA=SASHELP.CLASS;
RUN;
```

- 테이블1의 속성 결과

변수|유형|길이
-|-|-
AGE|숫자|8
HEIGHT|숫자|8
NAME|문자|8
SEX|문자|1
WEIGHT|숫자|8

- 테이블2는 아래와 같이 생성(새로운 테이블을 생성할 경우 컬럼 길이의 기본값은 길이가 8)

```SAS
DATA CLASS2;
INPUT NAME: $14. SEX $ AGE HEIGHT WEIGHT;
CARDS;
ABCDEFGHIJKLMN 1 17 207 90
OPQRSTUVWXYZAB 2 21 210 100
;
RUN;

PROC CONTENTS DATA=CLASS2;
RUN;
```

- 테이블2의 속성 결과

변수|유형|길이
-|-|-
AGE|숫자|8
HEIGHT|숫자|8
NAME|문자|14
SEX|문자|8
WEIGHT|숫자|8

- 이렇게 컬럼명은 동일하지만 길이가 다른 두 테이블을 합칠 경우, 기준이 되는 첫번째 지정된 테이블의 컬럼 길이에 따라 병합됨. 따라서 테이블1의 NAME컬럼의 길이인 8이 적용되므로 테이블2의 NAME컬럼은 길이8초과하는 부분은 짤린채로 병합됨

```SAS
DATA TEST;
SET
SASHELP.CLASS
CLASS2;
RUN;
```

- 병합된 테이블의 속성 결과

변수|유형|길이
-|-|-
AGE|숫자|8
HEIGHT|숫자|8
NAME|문자|8
SEX|문자|1
WEIGHT|숫자|8

#### 컬럼의 길이가 다른 경우 해결책1: 길이가 긴 테이블을 SET명령어에서 앞쪽에 나오도록 함

- 위 예시의 경우는 테이블2를 앞쪽에 테이블1을 뒤쪽에 오도록 하면 됨

```SAS
DATA TEST;
SET
CLASS2
SASHELP.CLASS;
RUN;
```

- 이렇게 병합된 테이블의 속성 결과

변수|유형|길이
-|-|-
AGE|숫자|8
HEIGHT|숫자|8
NAME|문자|14
SEX|문자|8
WEIGHT|숫자|8

#### 컬럼의 길이가 다른 경우 해결책 그외: 테이블을 생성할 때 두 테이블의 길이를 맞춰서 생성

#### 컬럼의 속성이 다른 경우

- 동일한 이름의 컬럼 간 속성의 차이가 있는 경우
- 예를 위해 테이블1은 SASHELP라이브러리의 CLASS테이블을 사용하고 테이블2는 추가 생성

```SAS
DATA CLASS2;
INPUT NAME: $14. SEX AGE HEIGHT WEIGHT;
CARDS;
ABCDEFGHIJKLMN 1 17 207 90
OPQRSTUVWXYZAB 2 21 210 100
;
RUN;
```

- 테이블2의 속성 결과

변수|유형|길이
-|-|-
AGE|숫자|8
HEIGHT|숫자|8
NAME|문자|14
SEX|숫자|8
WEIGHT|숫자|8

- 테이블1과 테이블2 병합

```SAS
DATA TEST;
SET
SASHELP.CLASS
CLASS2;
RUN;
```

- 이렇게 컬럼명이 같지만 컬럼 간 속성이 다른 경우(이 예시에서는 SEX컬럼이 한 테이블에서는 문자, 다른 테이블에서는 숫자임) 에러: 변수 SEX가 모두 문자와 숫자로 정의되었습니다. 이런 에러가 남.

#### 컬럼의 속성이 다른 경우 해결책1: 두 테이블 중 한 변수의 이름을 변경

- 하지만 이런 식으로 할 경우 각각의 컬럼이 생성됨

#### 컬럼의 속성이 다른 경우 해결책2: 테이블을 생성할 때 변수의 속성을 동일하게 만듬

- 숫자 변수를 문자 변수로 설정하는 것이 가장 일반적인 방법.

```SAS
DATA CLASS2;
INPUT NAME: $14. SEX $ AGE HEIGHT WEIGHT;
CARDS;
ABCDEFGHIJKLMN 1 17 207 90
OPQRSTUVWXYZAB 2 21 210 100
;
RUN;
```

- 이렇게 하면 제대로 병합됨

# 테이블 옆으로 붙이기 - MERGE

- MERGE XXX YYY: XXX테이블과 YYY테이블을 옆으로 병합
- 기준이 될 테이블1은 SASHELP라이브러리의 CLASS테이블 사용
- 옆으로 붙일 테이블은 다음과 같이 추가 생성


```SAS
DATA CLASS3;
INPUT NAME $ ADDR $ LINE;
CARDS;
JOHN SEOUL 5
MARCUS BUSAN 6
;
RUN;

DATA TEST;
MERGE
SASHELP.CLASS
CLASS3;
RUN;
```

- 그냥 이렇게 할 경우 제대로 병합되지 않음

#### 특정 값을 기준으로 결합하는 경우

- 두 테이블의 기준 컬럼은 이름이 같아야 함. 테이블1의 기준 컬럼 값과 동일한 기준 컬럼 값을 가진 테이블2의 데이터를 옆으로 붙임
- 예시에서는 기준 컬럼을 NAME으로 삼아 MERGE결합 진행
- ㅇ때, PROC SORT를 이용해 두 테이블의 기준 컬럼을 기준으로 정렬해야 함.

```SAS
DATA CLASS4;
INPUT NAME $ ADDR $ LINE;
CARDS;
John SEOUL 5
Barbara BUSAN 6
HARRY CITY 7
;
RUN;

PROC SORT DATA=SASHELP.CLASS OUT=CLASS;     /* SASHELP라이브러리의 CLASS테이블을 불러와 NAME컬럼을 정렬하고 CLASS라는 이름으로 저장 */
BY NAME;
RUN;

PROC SORT DATA=CLASS4;
BY NAME;
RUN;

DATA TEST;
MERGE     /* MERGE CLASS CLASS4; 이렇게 한 줄로 작성해도 됨 */
CLASS
CLASS4;
BY NAME;     /* MERGE결합 시 NAME컬럼을 기준으로 결합 */
RUN;
```